In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\79009\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### 1 Этап. Создание корпуса

Для создания корпуса был взят текст первых 6ти глав книги "Гарри Поттер и философский камень". Объем составляет около 23 тыс. слов. Был скачан .txt файл с сайта https://danielhp.narod.ru/books.html 

In [33]:
with open('harry_potter_stone.txt', 'r', encoding='cp1251') as file:
    text = file.read()

In [306]:
text[:1000]

'    \n   Джоан Кэтлин Роулинг \n   Гарри Поттер и философский камень \n    \n   Глава первая\n   Мальчик, который выжил.\n    \n   Мистер и миссис Дарсли, из дома номер четыре по Бирючинному проезду, могли бы \nс гордостью сказать, что они, слава богу, совершенно нормальные люди. Они были \nбы последними, от кого вы могли бы ожидать участия в чем-нибудь странном и \nтаинственном, потому что они совершенно не одобряли подобной чепухи. \n   Мистер Дарсли работал директором фирмы Граннингз, которая выпускала сверла. Он \nбыл крупный, крепкий мужчина с очень короткой шеей и очень большими усами. \nМиссис Дарсли была худой и светловолосой, зато ее шеи с лихвой хватило бы на \nдвоих. Этим подарком природы она усердно пользовалась, большую часть времени \nшпионя за соседями через садовые изгороди. У Дарсли рос сын Дадли, и они были \nуверены, что лучшего мальчика не сыщешь на всем свете. \n   У Дарсли было все что нужно, но кроме того, у них была тайна, и больше всего \nони боялись, что кто-

In [35]:
# разбиваем текст на главы, чтобы добавить в дальнейшем в датасет столбец с главой
chapter_pattern = r'Глава\s+(первая|вторая|третья|четвертая|пятая|шестая)'
chapters = re.split(chapter_pattern, text)

In [37]:
### Обрабатываем текст, удаляем лишние знаки (точки, запятые, дефисы итд)
### Кроме этого добавляем название и номер главы как столбцы датасета
data = []
for i in range(1, len(chapters), 2):
    chapter_number = i // 2 + 1 
    chapter_title = chapters[i].strip()  
    chapter_text = chapters[i + 1] if i + 1 < len(chapters) else ''
    sentences = sent_tokenize(chapter_text)

    if sentences:
        chapter_title = re.sub(r'[^А-Яа-яЁё0-9\s]', '', sentences[0]).strip().replace('\n', '').strip()

    for sentence in sentences[1:]:
        clean_sentence = re.sub(r'[^А-Яа-яЁё0-9\s-]', '', sentence).strip().replace('\n', '').strip()
        if clean_sentence:  
            data.append([chapter_number, chapter_title, clean_sentence])

df = pd.DataFrame(data, columns=['Номер главы', 'Название главы', 'Предложение'])

In [54]:
## Исправлен недочет с удалением дефисов в тексте. Проверка:
print(df["Предложение"].iloc[38])

Он пришел в бешенство увидев что некоторые совсем даже не молоды да этот мужчина вероятно старше его а одет в изумрудно-зеленую мантию


In [55]:
df.head(3)

,Номер главы,Название главы,Предложение
0,1,Мальчик который выжил,Мистер и миссис Дарсли из дома номер четыре по...
1,1,Мальчик который выжил,Они были бы последними от кого вы могли бы ожи...
2,1,Мальчик который выжил,Мистер Дарсли работал директором фирмы Граннин...


In [56]:
df.insert(0, 'Книга', 'Джоан Кэтлин Роулинг. Гарри Поттер и Философский камень.')
df.insert(0, 'Источник', 'https://danielhp.narod.ru/books.html')

In [57]:
df.head(3)

,Источник,Книга,Номер главы,Название главы,Предложение
0,https://danielhp.narod.ru/books.html,Джоан Кэтлин Роулинг. Гарри Поттер и Философск...,1,Мальчик который выжил,Мистер и миссис Дарсли из дома номер четыре по...
1,https://danielhp.narod.ru/books.html,Джоан Кэтлин Роулинг. Гарри Поттер и Философск...,1,Мальчик который выжил,Они были бы последними от кого вы могли бы ожи...
2,https://danielhp.narod.ru/books.html,Джоан Кэтлин Роулинг. Гарри Поттер и Философск...,1,Мальчик который выжил,Мистер Дарсли работал директором фирмы Граннин...


## 2 Этап. Делаем разбор каждого предложения и записываем в новый столбец в датасете

In [12]:
import stanza

In [74]:
# stanza.download('ru')

In [58]:
nlp = stanza.Pipeline('ru')

2024-10-29 20:10:35 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-10-29 20:10:35 INFO: Downloaded file to C:\Users\79009\stanza_resources\resources.json
2024-10-29 20:10:37 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |
| depparse  | syntagrus_charlm   |
| ner       | wikiner            |

2024-10-29 20:10:37 INFO: Using device: cpu
2024-10-29 20:10:37 INFO: Loading: tokenize
2024-10-29 20:10:37 INFO: Loading: pos
2024-10-29 20:10:37 INFO: Loading: lemma
2024-10-29 20:10:37 INFO: Loading: depparse
2024-10-29 20:10:38 INFO: Loading: ner
2024-10-29 20:10:39 INFO: Done loading processors!


In [59]:
### Для тестового написания дальнейшего кода брались первые 200 предложений из текста, чтобы ускорить процесс
### После проверки на тестовом датасете функции поиска делаем разбор для всего датасета
#df_test = df[:200].copy()
df_test = df.copy()

In [60]:
def analyze_sentence(sentence):
    doc = nlp(sentence)
    analysis = []
    for sentence in doc.sentences:
        for word in sentence.words:
            analysis.append({
                'text': word.text,         # Слово как в тексте
                'lemma': word.lemma,       # Лемма, т.е. начальная форма
                'pos': word.upos           # Часть речи (POS тег)
            })
    return analysis

In [61]:
import time
start_time = time.time()

df_test['Разбор'] = df_test['Предложение'].apply(analyze_sentence)

end_time = time.time()
print(f'Время разбора текста: {round((end_time - start_time)/60, 2)} минут')

Время разбора текста: 18.92 минут


In [62]:
df_test.iloc[10][4]

'Дарсли содрогались от мысли что скажут соседи если увидят Поттеров'

In [111]:
# пример хранения разбора предложения
df_test.iloc[10][5]

[{'text': 'Дарсли', 'lemma': 'дарсли', 'pos': 'NOUN'},
 {'text': 'содрогались', 'lemma': 'содрогать', 'pos': 'VERB'},
 {'text': 'от', 'lemma': 'от', 'pos': 'ADP'},
 {'text': 'мысли', 'lemma': 'мысль', 'pos': 'NOUN'},
 {'text': 'что', 'lemma': 'что', 'pos': 'PRON'},
 {'text': 'скажут', 'lemma': 'сказать', 'pos': 'VERB'},
 {'text': 'соседи', 'lemma': 'сосед', 'pos': 'NOUN'},
 {'text': 'если', 'lemma': 'если', 'pos': 'SCONJ'},
 {'text': 'увидят', 'lemma': 'видеть', 'pos': 'VERB'},
 {'text': 'Поттеров', 'lemma': 'Поттеров', 'pos': 'PROPN'}]

In [110]:
df_test.head(3)

,Источник,Книга,Номер главы,Название главы,Предложение,Разбор
0,https://danielhp.narod.ru/books.html,Джоан Кэтлин Роулинг. Гарри Поттер и Философск...,1,Мальчик который выжил,Мистер и миссис Дарсли из дома номер четыре по...,"[{'text': 'Мистер', 'lemma': 'мистер', 'pos': ..."
1,https://danielhp.narod.ru/books.html,Джоан Кэтлин Роулинг. Гарри Поттер и Философск...,1,Мальчик который выжил,Они были бы последними от кого вы могли бы ожи...,"[{'text': 'Они', 'lemma': 'они', 'pos': 'PRON'..."
2,https://danielhp.narod.ru/books.html,Джоан Кэтлин Роулинг. Гарри Поттер и Философск...,1,Мальчик который выжил,Мистер Дарсли работал директором фирмы Граннин...,"[{'text': 'Мистер', 'lemma': 'Мистер', 'pos': ..."


## 3 Этап. Функция поиска предложений

In [244]:
def parse_element(element):
    """
    Парсит один элемент из строки поиска и возвращает тип поиска (lemma, text, pos, lemma+pos, startswith, endswith, contains) и значение.
    """
    if '"' in element:  # text
        return 'text', element.strip('"')
    elif '+' in element:  # lemma+pos
        lemma, pos = element.split('+')
        return 'lemma+pos', (lemma, pos)
    elif element.isupper():  # pos
        return 'pos', element
    elif element.endswith('*') and not element.startswith('*'):  # начинается со *
        return 'startswith', element.rstrip('*')
    elif element.startswith('*') and not element.endswith('*'):  # заканчивается на *
        return 'endswith', element.lstrip('*')
    elif element.startswith('*') and element.endswith('*'):  # внутри *
        return 'contains', element.strip('*')
    else:  # lemma
        return 'lemma', element

def lemmatize_with_pos(word, pos):
    """
    Возвращает лемму и POS-тег слова, используя модель Stanza.
    """
    doc = nlp(word)
    token = doc.sentences[0].words[0]
    return token.lemma, token.upos

def process_query(query):
    """
    Преобразует элементы типа lemma и lemma+pos в их фактические леммы и POS-теги с помощью Stanza.
    """
    elements = query.split()
    processed_conditions = []
    for element in elements:
        search_type, value = parse_element(element)
        # Если это лемма, используем Stanza для получения леммы слова
        if search_type == 'lemma':
            lemma_value = lemmatize_word(value)
            processed_conditions.append((search_type, lemma_value))
        # Если это lemma+pos, используем Stanza для извлечения леммы и POS
        elif search_type == 'lemma+pos':
            lemma, pos = value
            lemma_value, actual_pos = lemmatize_with_pos(lemma, pos)
            processed_conditions.append((search_type, (lemma_value, actual_pos)))
        else:
            processed_conditions.append((search_type, value))
    return processed_conditions

def matches_word(word, search_type, value):
    """
    Проверяет, соответствует ли слово заданным условиям поиска
    text - то как слово встречается в тексте
    """
    if search_type == 'text':
        return word['text'] == value
    elif search_type == 'lemma':
        return word['lemma'] == value
    elif search_type == 'pos':
        return word['pos'] == value
    elif search_type == 'lemma+pos':
        lemma, pos = value
        return word['lemma'] == lemma and word['pos'] == pos
    elif search_type == 'startswith':
        return word['text'].startswith(value)
    elif search_type == 'endswith':
        return word['text'].endswith(value)
    elif search_type == 'contains':
        return value in word['text']
    return False

def find_sequence(words, conditions):
    """
    Проверяет, есть ли в предложении конструкция, соответствующая условиям поиска.
    """
    n = len(words)
    m = len(conditions)
    for i in range(n - m + 1):
        match = True
        for j in range(m):
            search_type, value = conditions[j]
            if not matches_word(words[i + j], search_type, value):
                match = False
                break
        if match:
            return i  # Возвращаем индекс начала найденной последовательности
    return -1 # если не нашли, то -1

def highlight_match(sentence, start_idx, condition_count):
    """
    Выделяет найденную конструкцию в предложении звездочками
    """
    words = sentence.split()
    highlighted_sentence = []
    for i, word in enumerate(words):
        if start_idx <= i < start_idx + condition_count:
            highlighted_sentence.append(f"*{word}*")
        else:
            highlighted_sentence.append(word)
    return " ".join(highlighted_sentence)

def search(df, query):
    conditions = process_query(query)
    
    results = []
    for _, row in df.iterrows():
        words = row['Разбор']
        start_idx = find_sequence(words, conditions)
        
        if start_idx != -1:
            # Выделяем найденное звездочками
            highlighted_sentence = highlight_match(row['Предложение'], start_idx, len(conditions))
            results.append((row['Номер главы'], row['Название главы'], highlighted_sentence))
    
    # Вывод 
    if not results:
        print(f"Предложений с конструкцией '{query}' в тексте не найдено.")
    else:
        print(f"Найденные предложения с конструкцией вида '{query}'\nв книге Гарри Поттер и философский камень (главы 1-6):")
        print("=" * 50)
        for i, (chapter, title, sentence) in enumerate(results, 1):
            print(f"{i}) Предложение из главы {chapter} \"{title}\"\n\"{sentence}\"")
            print("-" * 50)
        print(f"В тексте найдено {len(results)} предложений с конструкцией '{query}'.")
    return


## 4 Этап. Тестирование

Табличка с тегами UPOS

| POS тег | Описание                           |
|---------|------------------------------------|
| ADJ     | Прилагательное                     |
| ADP     | Предлог, послелог                  |
| ADV     | Наречие                            |
| AUX     | Вспомогательный глагол             |
| CCONJ   | Сочинительный союз                 |
| DET     | Детерминатив (артикль, указат. мест.) |
| INTJ    | Междометие                         |
| NOUN    | Существительное                    |
| NUM     | Числительное                       |
| PART    | Частица                            |
| PRON    | Местоимение                        |
| PROPN   | Собственное имя                    |
| PUNCT   | Знак препинания                    |
| SCONJ   | Подчинительный союз                |
| SYM     | Символ (например, &, %, +)         |
| VERB    | Глагол                             |
| X       | Другие части речи                  |


In [278]:
search(df_test, 'VERB')

Найденные предложения с конструкцией вида 'VERB'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Мистер и миссис Дарсли из дома номер четыре по Бирючинному проезду *могли* бы с гордостью сказать что они слава богу совершенно нормальные люди"
--------------------------------------------------
2) Предложение из главы 1 "Мальчик который выжил"
"Они были бы последними от кого вы *могли* бы ожидать участия в чем-нибудь странном и таинственном потому что они совершенно не одобряли подобной чепухи"
--------------------------------------------------
3) Предложение из главы 1 "Мальчик который выжил"
"Мистер Дарсли *работал* директором фирмы Граннингз которая выпускала сверла"
--------------------------------------------------
4) Предложение из главы 1 "Мальчик который выжил"
"Миссис Дарсли была худой и светловолосой зато ее шеи с лихвой *хватило* бы на двоих"
--------------------------------------------------
5) Предложение из главы 1 "М

In [279]:
# stanza обозначает нет как глагол
search(df_test, 'нет+VERB')

Найденные предложения с конструкцией вида 'нет+VERB'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Миссис Поттер приходилась миссис Дарсли сестрой которую та не видела вот уже несколько лет по правде говоря миссис Дарсли делала вид что у нее вообще *нет* сестры потому что сестра и ее никчемный муж были не похожи на Дарсли настолько насколько можно"
--------------------------------------------------
2) Предложение из главы 1 "Мальчик который выжил"
"*Нет* смысла беспокоить миссис Дарсли она всегда огорчается при упоминании о сестре"
--------------------------------------------------
3) Предложение из главы 1 "Мальчик который выжил"
"В конце концов они ведь делали вид что у нее вообще *нет* сестры"
--------------------------------------------------
4) Предложение из главы 1 "Мальчик который выжил"
"Говард *нет*"
--------------------------------------------------
5) Предложение из главы 1 "Мальчик который выжил"
"Его последней ут

In [280]:
# stanza обозначает нет как PART
search(df_test, 'нет+PART')

Найденные предложения с конструкцией вида 'нет+PART'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Миссис Поттер приходилась миссис Дарсли сестрой которую та не видела вот уже несколько лет по правде говоря миссис Дарсли делала вид что у нее вообще *нет* сестры потому что сестра и ее никчемный муж были не похожи на Дарсли настолько насколько можно"
--------------------------------------------------
2) Предложение из главы 1 "Мальчик который выжил"
"*Нет* смысла беспокоить миссис Дарсли она всегда огорчается при упоминании о сестре"
--------------------------------------------------
3) Предложение из главы 1 "Мальчик который выжил"
"В конце концов они ведь делали вид что у нее вообще *нет* сестры"
--------------------------------------------------
4) Предложение из главы 1 "Мальчик который выжил"
"Говард *нет*"
--------------------------------------------------
5) Предложение из главы 1 "Мальчик который выжил"
"Его последней ут

In [281]:
search(df_test, 'ADP мысль')

Найденные предложения с конструкцией вида 'ADP мысль'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Дарсли содрогались *от* *мысли* что скажут соседи если увидят Поттеров"
--------------------------------------------------
2) Предложение из главы 1 "Мальчик который выжил"
"Пытаясь собраться *с* *мыслями* он вошел в дом"
--------------------------------------------------
3) Предложение из главы 3 "Письмо от никого"
"Она разрешила Гарри посмотреть телевизор и угостила его шоколадным тортом вкус которого наводил *на* *мысль* что она хранит его долгие годы"
--------------------------------------------------
В тексте найдено 3 предложений с конструкцией 'ADP мысль'.


In [282]:
# лишние пробелы для писка не являются ограничением (из-за опечатки человек может ввести лишние пробелы, это нормально)
search(df_test, '  ADP     мысль   ')

Найденные предложения с конструкцией вида '  ADP     мысль   '
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Дарсли содрогались *от* *мысли* что скажут соседи если увидят Поттеров"
--------------------------------------------------
2) Предложение из главы 1 "Мальчик который выжил"
"Пытаясь собраться *с* *мыслями* он вошел в дом"
--------------------------------------------------
3) Предложение из главы 3 "Письмо от никого"
"Она разрешила Гарри посмотреть телевизор и угостила его шоколадным тортом вкус которого наводил *на* *мысль* что она хранит его долгие годы"
--------------------------------------------------
В тексте найдено 3 предложений с конструкцией '  ADP     мысль   '.


In [283]:
search(df_test, 'VERB сосед "если"')

Найденные предложения с конструкцией вида 'VERB сосед "если"'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Дарсли содрогались от мысли что *скажут* *соседи* *если* увидят Поттеров"
--------------------------------------------------
В тексте найдено 1 предложений с конструкцией 'VERB сосед "если"'.


In [284]:
search(df_test, 'verb сосед "если"')

Предложений с конструкцией 'verb сосед "если"' в тексте не найдено.


In [285]:
search(df_test, 'PART PRON') 

Найденные предложения с конструкцией вида 'PART PRON'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Несколько секунд он осознавал что *только* *что* видел потом резко повернул голову чтобы взглянуть опять"
--------------------------------------------------
2) Предложение из главы 1 "Мальчик который выжил"
"Если бы *не* *это* ему было бы трудно сконцентрироваться на сверлах этим утром"
--------------------------------------------------
3) Предложение из главы 1 "Мальчик который выжил"
"Он положил трубку и подергал себя за усы раздумывая *нет* *это* было глупо"
--------------------------------------------------
4) Предложение из главы 1 "Мальчик который выжил"
"Его *только* *что* обнял абсолютно незнакомый человек"
--------------------------------------------------
5) Предложение из главы 1 "Мальчик который выжил"
"*Разве* *это* нормальное поведение для кошки"
--------------------------------------------------
6) Предложение из 

In [286]:
search(df_test, 'чт* PART PRON') 

Найденные предложения с конструкцией вида 'чт* PART PRON'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Несколько секунд он осознавал *что* *только* *что* видел потом резко повернул голову чтобы взглянуть опять"
--------------------------------------------------
2) Предложение из главы 2 "Исчезнувшее стекло"
"Дело в том что вокруг Гарри часто происходили странные вещи и было бесполезно убеждать Дарсли в том *что* *не* *он* их причина"
--------------------------------------------------
3) Предложение из главы 4 "Хранитель ключей"
"Я запрещаю вам рассказывать мальчику *что* *бы* *то* ни было"
--------------------------------------------------
4) Предложение из главы 5 "Диагон аллея"
"У Гарри было забавное чувство *что* *только* *он* и Хагрид на самом деле его видят"
--------------------------------------------------
В тексте найдено 4 предложений с конструкцией 'чт* PART PRON'.


In [287]:
search(df_test, '*чт* PART PRON') 

Найденные предложения с конструкцией вида '*чт* PART PRON'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Несколько секунд он осознавал *что* *только* *что* видел потом резко повернул голову чтобы взглянуть опять"
--------------------------------------------------
2) Предложение из главы 2 "Исчезнувшее стекло"
"Дело в том что вокруг Гарри часто происходили странные вещи и было бесполезно убеждать Дарсли в том *что* *не* *он* их причина"
--------------------------------------------------
3) Предложение из главы 4 "Хранитель ключей"
"Я запрещаю вам рассказывать мальчику *что* *бы* *то* ни было"
--------------------------------------------------
4) Предложение из главы 5 "Диагон аллея"
"У Гарри было забавное чувство *что* *только* *он* и Хагрид на самом деле его видят"
--------------------------------------------------
В тексте найдено 4 предложений с конструкцией '*чт* PART PRON'.


In [288]:
search(df_test, '*здум* PART PRON') 

Найденные предложения с конструкцией вида '*здум* PART PRON'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Он положил трубку и подергал себя за усы *раздумывая* *нет* *это* было глупо"
--------------------------------------------------
В тексте найдено 1 предложений с конструкцией '*здум* PART PRON'.


In [289]:
search(df_test, '*здум*PART PRON') 

Предложений с конструкцией '*здум*PART PRON' в тексте не найдено.


In [290]:
search(df_test, 'VERB') 

Найденные предложения с конструкцией вида 'VERB'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Мистер и миссис Дарсли из дома номер четыре по Бирючинному проезду *могли* бы с гордостью сказать что они слава богу совершенно нормальные люди"
--------------------------------------------------
2) Предложение из главы 1 "Мальчик который выжил"
"Они были бы последними от кого вы *могли* бы ожидать участия в чем-нибудь странном и таинственном потому что они совершенно не одобряли подобной чепухи"
--------------------------------------------------
3) Предложение из главы 1 "Мальчик который выжил"
"Мистер Дарсли *работал* директором фирмы Граннингз которая выпускала сверла"
--------------------------------------------------
4) Предложение из главы 1 "Мальчик который выжил"
"Миссис Дарсли была худой и светловолосой зато ее шеи с лихвой *хватило* бы на двоих"
--------------------------------------------------
5) Предложение из главы 1 "М

In [294]:
search(df_test, '"verb" сосед "если"')

Предложений с конструкцией '"verb" сосед "если"' в тексте не найдено.


In [295]:
search(df_test, 'сосед "ес*"')

Предложений с конструкцией 'сосед "ес*"' в тексте не найдено.


In [296]:
search(df_test, '"сосед ес*"')

Предложений с конструкцией '"сосед ес*"' в тексте не найдено.


In [300]:
search(df_test, '"соседи если"')

Найденные предложения с конструкцией вида '"соседи если"'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Дарсли содрогались от мысли что скажут *соседи* *если* увидят Поттеров"
--------------------------------------------------
В тексте найдено 1 предложений с конструкцией '"соседи если"'.


In [298]:
search(df_test, 'сосед если увидеть')

Найденные предложения с конструкцией вида 'сосед если увидеть'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Дарсли содрогались от мысли что скажут *соседи* *если* *увидят* Поттеров"
--------------------------------------------------
В тексте найдено 1 предложений с конструкцией 'сосед если увидеть'.


In [264]:
search(df_test, 'VERB от')

Найденные предложения с конструкцией вида 'VERB от'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Дарсли *содрогались* *от* мысли что скажут соседи если увидят Поттеров"
--------------------------------------------------
2) Предложение из главы 1 "Мальчик который выжил"
"Он не казался *расстроенным* *от* того что его чуть не уронили"
--------------------------------------------------
3) Предложение из главы 2 "Исчезнувшее стекло"
"Наконец *устав* *от* усилий он медленно произнес Тогда у меня будет тридцать тридцать Тридцать девять мой сладкий - сказала тетя Петуния"
--------------------------------------------------
4) Предложение из главы 2 "Исчезнувшее стекло"
"Однажды тетя Петуния *устав* *от* того что Гарри приходит из парикмахерской как будто никогда там не был взяла пару кухонных ножниц и постригла его почти наголо оставив только челку чтобы спрятать этот ужасный шрам"
--------------------------------------------------
5

In [265]:
search(df_test, 'VERB от мысли')

Найденные предложения с конструкцией вида 'VERB от мысли'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Дарсли *содрогались* *от* *мысли* что скажут соседи если увидят Поттеров"
--------------------------------------------------
В тексте найдено 1 предложений с конструкцией 'VERB от мысли'.


In [266]:
search(df_test, 'VERB "от мысли"')

Найденные предложения с конструкцией вида 'VERB "от мысли"'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Дарсли *содрогались* *от* *мысли* что скажут соседи если увидят Поттеров"
--------------------------------------------------
В тексте найдено 1 предложений с конструкцией 'VERB "от мысли"'.


In [267]:
search(df_test, 'ска* *ос*')

Найденные предложения с конструкцией вида 'ска* *ос*'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Дарсли содрогались от мысли что *скажут* *соседи* если увидят Поттеров"
--------------------------------------------------
В тексте найдено 1 предложений с конструкцией 'ска* *ос*'.


In [268]:
search(df_test, 'скажут+VERB сосед+NOUN')

Найденные предложения с конструкцией вида 'скажут+VERB сосед+NOUN'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Дарсли содрогались от мысли что *скажут* *соседи* если увидят Поттеров"
--------------------------------------------------
В тексте найдено 1 предложений с конструкцией 'скажут+VERB сосед+NOUN'.


In [277]:
search(df_test, 'закончилась но ему')

Найденные предложения с конструкцией вида 'закончилась но ему'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 3 "Письмо от никого"
"Гарри был рад что школа *закончилась* *но* *ему* не удалось избавиться от банды Дадли которая бывала в доме каждый день"
--------------------------------------------------
В тексте найдено 1 предложений с конструкцией 'закончилась но ему'.


In [301]:
# более 3х элементов тоже ищет
search(df_test, 'закончилась но ему не удалось избавиться')

Найденные предложения с конструкцией вида 'закончилась но ему не удалось избавиться'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 3 "Письмо от никого"
"Гарри был рад что школа *закончилась* *но* *ему* *не* *удалось* *избавиться* от банды Дадли которая бывала в доме каждый день"
--------------------------------------------------
В тексте найдено 1 предложений с конструкцией 'закончилась но ему не удалось избавиться'.


In [304]:
# более 3х элементов тоже ищет
search(df_test, '*кон* "но ему" не удалось+VERB VERB')

Найденные предложения с конструкцией вида '*кон* "но ему" не удалось+VERB VERB'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 3 "Письмо от никого"
"Гарри был рад что школа *закончилась* *но* *ему* *не* *удалось* *избавиться* от банды Дадли которая бывала в доме каждый день"
--------------------------------------------------
В тексте найдено 1 предложений с конструкцией '*кон* "но ему" не удалось+VERB VERB'.


In [305]:
search(df_test, '"что" "случится"')

Найденные предложения с конструкцией вида '"что" "случится"'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Они с ужасом представляли *что* *случится* если кто-то вдруг узнает о Поттерах"
--------------------------------------------------
В тексте найдено 1 предложений с конструкцией '"что" "случится"'.


In [293]:
# так тоже можно искать
search(df_test, '"что случится"')

Найденные предложения с конструкцией вида '"что случится"'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Они с ужасом представляли *что* *случится* если кто-то вдруг узнает о Поттерах"
--------------------------------------------------
В тексте найдено 1 предложений с конструкцией '"что случится"'.


In [276]:
search(df_test, 'VERB ADP NOUN')

Найденные предложения с конструкцией вида 'VERB ADP NOUN'
в книге Гарри Поттер и философский камень (главы 1-6):
1) Предложение из главы 1 "Мальчик который выжил"
"Дарсли *содрогались* *от* *мысли* что скажут соседи если увидят Поттеров"
--------------------------------------------------
2) Предложение из главы 1 "Мальчик который выжил"
"Мистер Дарсли мурлыкал себе под нос пока *выбирал* *на* *работу* свой самый скучный галстук а миссис Дарсли радостно болтала водружая орущего Дадли на высокий стул"
--------------------------------------------------
3) Предложение из главы 1 "Мальчик который выжил"
"Никто из них не заметил большой золотисто-коричневой совы промелькнувшей за *окном*"
--------------------------------------------------
4) Предложение из главы 1 "Мальчик который выжил"
"Ах ты разбойник - усмехнулся мистер Дарсли *выходя* *из* *дома*"
--------------------------------------------------
5) Предложение из главы 1 "Мальчик который выжил"
"Он *сел* *в* *машину* и выехал на улицу